# Game Cosmetics vs. Player Performance

### Do certain cosmetics have an impact on player performance in video games?

I have been wondering this question for AGES. Every multiplayer game has a phenomena called "Sweat skins," where higher performing players tend to flock to certain cosmetics, and thus it gives itself the name. When these certain cosmetics start to gain a reputation, general player attitude tends to change toward that cosmetic as well.

The question I'm trying to solve is whether this community phenomena has real effects on players' performance. Most game companies claim that the cosmetics they sell have no impact on gameplay. I want to put that claim to the test.


In [ ]:
print("Hello World!")
